Utilizando o Python para criar um script de carregamento para a "Produtos"

In [1]:
import pandas as pd

Mostrando o conteudo do CSV (Apenas os dois primeiros)

In [2]:
df = pd.read_csv('dim_produtos.csv')
df.head(2)

,id_produto,nome,categoria,preco_unitario,id_planta
0,1,Rolamento Aut-001,Automotivo,292.61,8
1,2,Rolamento Aut-002,Automotivo,173.40,3


Visualizando os metadados do nosso arquivo, para checagem de tipo de dados etc

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_produto      50 non-null     int64  
 1   nome            50 non-null     object 
 2   categoria       50 non-null     object 
 3   preco_unitario  50 non-null     float64
 4   id_planta       50 non-null     int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 2.1+ KB


Primeiro modo, inserção manual, apenas cria um script de inserção.

In [4]:
with open('inserts_produtos.sql', 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        nome = row['nome'].replace("'", "''")
        categoria = row['categoria'].replace("'", "''")
        preco_unitario = row['preco_unitario']
        
        insert_sql = (
            "INSERT INTO produtos (id_produto, nome, categoria, preco_unitario, id_planta) "
            "VALUES ({}, '{}', '{}', {}, {});\n".format(row['id_produto'], nome, categoria, preco_unitario, row['id_planta'])
        )
        f.write(insert_sql)

print("Arquivo 'insert_produtos.sql' gerado com sucesso!")


Arquivo 'insert_produtos.sql' gerado com sucesso!


Segundo modo, inserção direta no banco de dados, executa os comandos de insert diretamente.

In [ ]:
# Primeiramente, vou utilizar uma biblioteca para conectar ao MySQL e inserir os dados do DataFrame.

import mysql.connector

# Função para inserir dados no MySQL, definindo o nome da tabela como 'produtos' e a conexão com o banco de dados.
def insert_data_to_mysql(df, table_name='produtos'):
    connection_params = {
        'host': 'localhost',
        'user': 'root',
        'password': 'pw4dummy',
        'database': 'prod_nsk'
    }
    data_to_insert = []  # declarar fora do try para retornar depois

    try:
        # Conectar ao banco de dados MySQL
        conn = mysql.connector.connect(**connection_params)
        # criar um cursor para executar comandos SQL
        cursor = conn.cursor()

        # SQL para inserir dados na tabela
        # Usando f-string para facilitar a inserção de dados
        sql = f"INSERT INTO {table_name} (id_produto, nome, categoria, preco_unitario, id_planta) VALUES (%s, %s, %s, %s, %s)"

        # Iterar sobre o DataFrame e preparar os dados para inserção
        # Usando executemany para inserir múltiplas linhas de uma vez
        for _, row in df.iterrows():
            values = (
                row['id_produto'],
                row['nome'],
                row['categoria'],
                row['preco_unitario'],
                row['id_planta']
            )
            data_to_insert.append(values)

        cursor.executemany(sql, data_to_insert)
        conn.commit()

        #mensagem de sucesso
        print(f"{cursor.rowcount} registros inseridos com sucesso na tabela {table_name}.")
    
    except mysql.connector.Error as err:
        # mensagem de erro
        print(f"Erro MySQL: {err}")
    
    # fechar o cursor e a conexão
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
    
    return data_to_insert  # retornando aqui

# Executar e capturar os dados
data_to_insert = insert_data_to_mysql(df)


8 registros inseridos com sucesso na tabela produtos.
